# Hello World

This is a self contained guide on how to build a simple app and component spec
and launch it via two different schedulers.

See the [Quickstart](../quickstart.rst) for how to install TorchX locally before
following this example.

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

> **NOTE**: This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.

In [1]:
%%writefile my_app.py

import sys
import argparse

def main(user: str) -> None:
    print(f"Hello, {user}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Hello world app"
    )
    parser.add_argument(
        "--user",
        type=str,
        help="the person to greet",
        required=True,
    )
    args = parser.parse_args(sys.argv[1:])

    main(args.user)

Writing my_app.py


Now that we have an app we can write the component file for it. This
function allows us to reuse and share our app in a user friendly way.

We can use this component from the `torchx` cli or programmatically as part of a
pipeline.

In [2]:
%%writefile my_component.py

import torchx.specs as specs

def greet(user: str, image: str = "my_app:latest") -> specs.AppDef:
    """
    Echos a message to stdout (calls /bin/echo)

    Args:
        user: name of the person to greet
        image: image to use
    """
    return specs.AppDef(
        name="hello_world",
        roles=[
            specs.Role(
                name="greeter",
                image=image,
                entrypoint="python",
                args=[
                    "-m", "my_app",
                    "--user", user,
                ],
            )
        ],
    )

Writing my_component.py


Once we write our component, we can then call it via `torchx run`. The
`local_cwd` scheduler executes the component relative to the current directory.

In [3]:
%%sh
torchx run --scheduler local_cwd my_component.py:greet --user "your name"

Hello, your name!
local_cwd://torchx/hello_world_4e9b64f2


torchx 2021-10-15 20:44:44 INFO     Waiting for the app to finish...
torchx 2021-10-15 20:44:45 INFO     Job finished: SUCCEEDED


If we want to run in other environments, we can build a Docker container so we
can run our component in Docker enabled environments such as Kubernetes or via
the local Docker scheduler.

> **NOTE:** this requires Docker installed and won't work in environments such as
Google Colab.

In [4]:
%%writefile Dockerfile

FROM ghcr.io/pytorch/torchx:0.1.0rc1

ADD my_app.py .

Writing Dockerfile


Once we have the Dockerfile created we can create our docker image.

In [5]:
%%sh
docker build -t my_app:latest -f Dockerfile .


Step 1/2 : FROM ghcr.io/pytorch/torchx:0.1.0rc1
0.1.0rc1: Pulling from pytorch/torchx
4bbfd2c87b75: Pulling fs layer
d2e110be24e1: Pulling fs layer
889a7173dcfe: Pulling fs layer
6009a622672a: Pulling fs layer
143f80195431: Pulling fs layer
eccbe17c44e1: Pulling fs layer
c4cd934092e7: Pulling fs layer
d61c70aa1e93: Pulling fs layer
001b034ddad5: Pulling fs layer
182a871c314a: Pulling fs layer
e183586e7d41: Pulling fs layer
c2fb7190f97a: Pulling fs layer
6de368d18056: Pulling fs layer
957f2522dff4: Pulling fs layer
6009a622672a: Waiting
143f80195431: Waiting
eccbe17c44e1: Waiting
c4cd934092e7: Waiting
d61c70aa1e93: Waiting
001b034ddad5: Waiting
182a871c314a: Waiting
e183586e7d41: Waiting
c2fb7190f97a: Waiting
6de368d18056: Waiting
957f2522dff4: Waiting
d2e110be24e1: Verifying Checksum
d2e110be24e1: Download complete
889a7173dcfe: Verifying Checksum
889a7173dcfe: Download complete
4bbfd2c87b75: Verifying Checksum
4bbfd2c87b75: Download complete
6009a622672a: Verifying Checksum
6009a6226

We can then launch it on the local scheduler.

In [6]:
%%sh
torchx run --scheduler local_docker my_component.py:greet --image "my_app:latest" --user "your name"

Hello, your name!
local_docker://torchx/hello_world_05a092fb


Error response from daemon: pull access denied for my_app, repository does not exist or may require 'docker login': denied: requested access to the resource is denied
torchx 2021-10-15 20:47:01 WARNING  failed to fetch image my_app:latest, falling back to local: Command '['docker', 'pull', 'my_app:latest']' returned non-zero exit status 1.
torchx 2021-10-15 20:47:01 INFO     Waiting for the app to finish...
torchx 2021-10-15 20:47:02 INFO     Job finished: SUCCEEDED


If you have a Kubernetes cluster you can use the Kubernetes scheduler to launch
this on the cluster instead.